In [ ]:
!pip install -Uqq fastai

In [ ]:
!pip install duckduckgo-search

In [ ]:
from duckduckgo_search import DDGS
from fastcore.all import *

def search_images(term, max_images=30):
    print(f"Finding '{term}'")
    with DDGS() as ddgs:
        search_results = ddgs.images(keywords=term)
        image_urls = [next(search_results).get("image") for _ in range(max_images)]
        return L(image_urls)


In [ ]:
urls = search_images('lunar crater', max_images=1)
urls[0]

In [ ]:
from fastdownload import download_url
dest = 'lunar_crater.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

In [ ]:
download_url(search_images('Lunar rock', max_images=1)[0], 'lunar_rock.jpg', show_progress=False)
Image.open('lunar_rock.jpg').to_thumb(256,256)

In [ ]:
from pathlib import Path
searches = 'lunar crater', 'lunar rock'
path = Path('crater_or_not')
from time import sleep

for o in searches:
    dest = (path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(o))
    
    resize_images(path/o, max_size=400, dest=path/o)

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
get_items=get_image_files,
splitter=RandomSplitter(valid_pct=0.2, seed=42),
get_y=parent_label,
item_tfms=[Resize(192, method='squish')]
).dataloaders(path, bs=32)

dls.show_batch(max_n=6)
    
    

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(3)

In [ ]:
is_lunar_crater, _, probs = learn.predict(PILImage.create('lunar_crater.jpg'))

print(f"This is a: {is_lunar_crater}")
print(f"Probability it's a crater: {probs[0]:.4f}")

In [ ]:
import ipywidgets as widgets

uploader = widgets.FileUpload()
uploader

In [ ]:
img = PILImage.create(uploader.data[0])
is_lunar_crater, _, probs = learn.predict(img)

print(f"This is a: {is_lunar_crater}")
print(f"Probability it's a crater: {probs[0]:.4f}")